In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,64,2024-03-09,3,112.000000,95.509524,0.532984,10.531746,24.336508,0.187921,115.422222,...,11.152459,24.378689,0.216492,120.690164,0,3,5,2,4,2024
1,64,2024-03-09,1,118.650794,99.661905,0.563540,10.582540,22.153968,0.198286,119.044444,...,13.056452,24.790323,0.196468,112.251613,0,3,5,2,7,2024
2,64,2024-03-09,-1,112.000000,96.707937,0.536302,10.777778,24.553968,0.175302,115.200000,...,11.975806,21.546774,0.167758,110.217742,0,3,5,2,3,2024
3,63,2024-03-09,-2,120.806452,97.235484,0.576516,10.541935,24.408065,0.195210,123.414516,...,12.657143,24.819048,0.239317,118.853968,0,3,5,2,2,2024
4,64,2024-03-09,-2,112.428571,100.914286,0.532587,12.603175,22.728571,0.172683,111.165079,...,12.046774,27.356452,0.183968,118.522581,0,3,5,2,27,2024
5,64,2024-03-09,-1,117.666667,99.357143,0.543746,12.907937,28.738095,0.211492,117.493651,...,10.755556,25.803175,0.176365,119.058730,0,3,5,3,29,2024
6,64,2024-03-09,-2,114.111111,98.668254,0.541746,11.263492,24.912698,0.182889,114.958730,...,12.562295,26.695082,0.194754,110.259016,0,3,5,2,28,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-09,CHI,LAC,3.200000,1.370370,0
1,2024-03-09,DAL,DET,1.327869,3.450000,1
2,2024-03-09,BRK,CHO,1.420168,2.950000,1
3,2024-03-09,BOS,PHO,1.458716,2.800000,0
4,2024-03-09,SAS,GSW,4.600000,1.212766,0
5,2024-03-09,UTA,DEN,6.500000,1.125000,0
6,2024-03-09,TOR,POR,1.704225,2.200000,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-09,CHI,LAC,3.200000,1.370370,0
1,2024-03-09,DAL,DET,1.327869,3.450000,1
2,2024-03-09,BRK,CHO,1.420168,2.950000,1
3,2024-03-09,BOS,PHO,1.458716,2.800000,0
4,2024-03-09,SAS,GSW,4.600000,1.212766,0
5,2024-03-09,UTA,DEN,6.500000,1.125000,0
6,2024-03-09,TOR,POR,1.704225,2.200000,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-09,CHI,LAC,3.200000,1.370370,0
1,2024-03-09,DAL,DET,1.327869,3.450000,1
2,2024-03-09,BRK,CHO,1.420168,2.950000,1
3,2024-03-09,BOS,PHO,1.458716,2.800000,0
4,2024-03-09,SAS,GSW,4.600000,1.212766,0
5,2024-03-09,UTA,DEN,6.500000,1.125000,0
6,2024-03-09,TOR,POR,1.704225,2.200000,1


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-09,CHI,LAC,3.200000,1.370370,0
1,2024-03-09,DAL,DET,1.327869,3.450000,1
2,2024-03-09,BRK,CHO,1.420168,2.950000,1
3,2024-03-09,BOS,PHO,1.458716,2.800000,0
4,2024-03-09,SAS,GSW,4.600000,1.212766,0
5,2024-03-09,UTA,DEN,6.500000,1.125000,0
6,2024-03-09,TOR,POR,1.704225,2.200000,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-09,CHI,LAC,3.200000,1.370370,0.0
1,2024-03-09,DAL,DET,1.327869,3.450000,1.0
2,2024-03-09,BRK,CHO,1.420168,2.950000,1.0
3,2024-03-09,BOS,PHO,1.458716,2.800000,1.0
4,2024-03-09,SAS,GSW,4.600000,1.212766,0.0
5,2024-03-09,UTA,DEN,6.500000,1.125000,0.0
6,2024-03-09,TOR,POR,1.704225,2.200000,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-09,CHI,LAC,3.200000,1.370370,0.0
1,2024-03-09,DAL,DET,1.327869,3.450000,1.0
2,2024-03-09,BRK,CHO,1.420168,2.950000,1.0
3,2024-03-09,BOS,PHO,1.458716,2.800000,1.0
4,2024-03-09,SAS,GSW,4.600000,1.212766,0.0
5,2024-03-09,UTA,DEN,6.500000,1.125000,0.0
6,2024-03-09,TOR,POR,1.704225,2.200000,1.0
